In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [3]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10)) # wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5) # + noise
    return series[..., np.newaxis].astype(np.float32)

In [4]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [5]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(25, return_sequences = True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(25),
    keras.layers.Dense(1)
])

In [6]:
model.compile(loss="mse", optimizer = 'adam', metrics = [tf.keras.metrics.MeanSquaredError()])

In [7]:
history = model.fit(X_train, y_train, epochs = 10, validation_data = (X_valid, y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/10
7000/7000 [==============================] - 9s 1ms/sample - loss: 0.0284 - mean_squared_error: 0.0284 - val_loss: 0.0070 - val_mean_squared_error: 0.0070
Epoch 2/10
7000/7000 [==============================] - 7s 1ms/sample - loss: 0.0051 - mean_squared_error: 0.0051 - val_loss: 0.0049 - val_mean_squared_error: 0.0049
Epoch 3/10
7000/7000 [==============================] - 7s 1ms/sample - loss: 0.0036 - mean_squared_error: 0.0036 - val_loss: 0.0041 - val_mean_squared_error: 0.0041
Epoch 4/10
7000/7000 [==============================] - 7s 1ms/sample - loss: 0.0032 - mean_squared_error: 0.0032 - val_loss: 0.0031 - val_mean_squared_error: 0.0031
Epoch 5/10
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0030 - mean_squared_error: 0.0030 - val_loss: 0.0030 - val_mean_squared_error: 0.0030
Epoch 6/10
7000/7000 [==============================] - 7s 1ms/sample - loss: 0.0030 - mean_squared_error: 0.0030 - val_lo

In [9]:
y_pred = model.predict(X_test)

In [26]:
MSE = np.mean(tf.keras.losses.mean_squared_error(y_test, y_pred))

print("MSE for test data: " + str(MSE))

MSE for test data: 0.0032853198
